In [28]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import corenlp as cnlp

In [29]:
def treat_text(text_, remove=['\n', '\r', '\t', '\l', '\f', '\p']):
    text = text_
    if text is None:
        return ''
    for r in remove:
        text = text.replace(r, '')
    return text.strip()

### Harem

In [30]:
%%time

DIVIDERS = ['.', ';', '?', '!']
CODE_SENTENCE = 0
words = []
pos = []
graphics = []
sentences_code = []
bio_classes = []
biose_classes = []
classes = []
def graphic_feature(word):
    if word.isupper():
        return 4
    if word.istitle():
        return 3
    if word.isdigit():
        return 2
    if not word.isalpha():
        return 1
    return 0;
    

def extract_features(text_, class_=None):
    #global words, pos, graphics, sentences_code, bio_classes, biose_classes, classes
    global CODE_SENTENCE
    import time
    while True:
        _text = treat_text(text_)
        try:
            words_, pos_ = cnlp.tokens_pos(_text)
            break
        except:
            print("Connection refused by the server.. Sleep for 1 seconds")
            print('=', text_)
            time.sleep(1)
    if len(words_) != len(pos_):
        print('WARNING:', len(words_), '!=', len(pos_), '\n', words_, pos_)
    else:
        for i in range(len(words_)):
            w = words_[i]
            p = pos_[i]
            words.append(w)
            pos.append(p)
            graphics.append(graphic_feature(w))
            sentences_code.append(CODE_SENTENCE)
            classes.append(class_)
            if class_ is not None:
                if i == 0:
                    bio_classes.append('B')
                    if len(words_) == 1:
                        biose_classes.append('S')
                    else:
                        biose_classes.append('B')
                else:
                    bio_classes.append('I')
                    if i+1 == len(words_):
                        biose_classes.append('E')
                    else:
                        biose_classes.append('I')
            else:
                bio_classes.append('O')
                biose_classes.append('O')
            if w.strip() in DIVIDERS:
                CODE_SENTENCE += 1


path_harem = "data/CDSegundoHAREM.xml"

root = ET.parse(path_harem).getroot()
docs = root.findall('DOC')


for i_, doc in enumerate(docs):
    print(round((i_+1)/len(docs)*100,2), '%')
    for j_, p in enumerate(doc.findall('P')):
        extract_features(p.text)
        for t in p:
            if t.tag == 'EM':
                if 'CATEG' in t.attrib:
                    extract_features(t.text, t.attrib['CATEG'].split('|')[0])
                else:
                    extract_features(t.text, 'OUTRO')
            elif t.tag == 'ALT':
                if 'CATEG' in t[0].attrib:
                    extract_features(t[0].text, t[0].attrib['CATEG'].split('|')[0])
                else:
                    extract_features(t[0].text, 'OUTRO')
            elif t.tag == 'OMITIDO':
                text = t.text
                if text is None:
                    text = ''
                for tt in t:
                    if tt.text is not None:
                        text += ' ' + tt.text
                    b_ = False
                    if tt.tag == 'ALT':
                        b_ = True
                        for ttt in tt:
                            text += ' ' + ttt.text
                            if ttt.tail is not None:
                                text += ' ' + ttt.tail
                    if tt.tail is not None:
                        text += ' ' + tt.tail
                    if b_:
                        break
                extract_features(text)
            if t.tail is not None:
                extract_features(t.tail)
        CODE_SENTENCE += 1        
            
    
    #sentences = cnlp.sentences_tokens_pos(doc.text)
    #print(doc.text)
    
    


0.78 %
1.55 %
2.33 %
3.1 %
3.88 %
4.65 %
5.43 %
6.2 %
6.98 %
7.75 %
8.53 %
9.3 %
10.08 %
10.85 %
11.63 %
12.4 %
13.18 %
13.95 %
14.73 %
15.5 %
16.28 %
17.05 %
17.83 %
18.6 %
19.38 %
20.16 %
20.93 %
21.71 %
22.48 %
23.26 %
24.03 %
24.81 %
25.58 %
26.36 %
27.13 %
27.91 %
28.68 %
29.46 %
30.23 %
31.01 %
31.78 %
32.56 %
33.33 %
34.11 %
34.88 %
35.66 %
36.43 %
37.21 %
37.98 %
38.76 %
39.53 %
40.31 %
41.09 %
41.86 %
42.64 %
43.41 %
44.19 %
44.96 %
45.74 %
46.51 %
47.29 %
48.06 %
48.84 %
49.61 %
50.39 %
51.16 %
51.94 %
52.71 %
53.49 %
54.26 %
55.04 %
55.81 %
56.59 %
57.36 %
58.14 %
Connection refused by the server.. Sleep for 1 seconds
= 
      Paralelamente, as tropas portuguesas conseguiram expulsar os holandeses do
      
Connection refused by the server.. Sleep for 1 seconds
= 
      Paralelamente, as tropas portuguesas conseguiram expulsar os holandeses do
      
Connection refused by the server.. Sleep for 1 seconds
= 
      Paralelamente, as tropas portuguesas conseguiram expulsar os h

Connection refused by the server.. Sleep for 1 seconds
= Metodologia de Atribuição de Licenças de Emissão - PNALE 2005-2007
Connection refused by the server.. Sleep for 1 seconds
= Metodologia de Atribuição de Licenças de Emissão - PNALE 2005-2007
Connection refused by the server.. Sleep for 1 seconds
= Metodologia de Atribuição de Licenças de Emissão - PNALE 2005-2007
Connection refused by the server.. Sleep for 1 seconds
=  
Connection refused by the server.. Sleep for 1 seconds
=  
Connection refused by the server.. Sleep for 1 seconds
=  do 
Wall time: 3min 38s


In [31]:
df = pd.DataFrame({'Word':words, 'Pos':pos, 'Graphic':graphics, 
                   'sentence_code':sentences_code, 'bio': bio_classes,
                   'biose': biose_classes, 'class': classes})

In [32]:
df

,Graphic,Pos,Word,bio,biose,class,sentence_code
0,3,NOUN,Fatores,O,O,None,0
1,3,PNOUN,Demográficos,O,O,None,0
2,0,CONJ,e,O,O,None,0
3,3,PNOUN,Econômicos,O,O,None,0
4,3,PNOUN,Subjacentes,O,O,None,0
5,4,DET,A,O,O,None,1
6,0,NOUN,revolta,O,O,None,1
7,0,ADJ,histórica,O,O,None,1
8,0,VERB,produz,O,O,None,1
9,0,ADV,normalmente,O,O,None,1


In [33]:
df.to_csv('harem.csv', index=False)

In [10]:
print(len(words), len(pos), len(graphics), len(sentences_code), len(bio_classes), len(biose_classes), len(classes))

86074 86074 86074 86074 15311 15311 86074


In [ ]:
for doc in root.iterfind('DOC'):
    #print(doc.attrib)
    
    labeled_tokens = []
    
    labeled_tokens += [('O', token) for token in nltk.word_tokenize(doc.text)]
    
    for em in doc.iterfind('EM'):
        print(em.tag)
        
        labeled_tokens += [(em.attrib['CATEG'], token) for token in nltk.word_tokenize(em.text)]
        
        labeled_tokens += [('O', token) for token in nltk.word_tokenize(em.tail)]
        
    #print(labeled_tokens)
        

In [ ]:
x,y = cnlp.tokens_pos(', e a ele n?o se deveria atribuir pouca responsabilidade pelo fato de que seres urbanos de todo o mundo agora vivam numa paisagem midiática onde enxutas batidas eletrônicas servem como trilha sonora para tudo, desde idas ao shopping e suas experiências com drogas, a noites em casa em frente da tv.')

In [ ]:
b = []

def gaz():
    global b
    b.append(1)
    
gaz()